In [1]:
import os
#import sys; sys.path.append("~/HHbbgg_ETH_devel/Training/python") # to load packages
import sys; sys.path.append("/home/ovtin/cernbox/HHggbb/HHbbgg_ETH_devel/Training/python") # to load packages
import training_utils as utils
import numpy as np
reload(utils)
import preprocessing_utils as preprocessing
reload(preprocessing)
import plotting_utils as plotting
reload(plotting)
import optimization_utils as optimization
reload(optimization)
import postprocessing_utils as postprocessing
reload(postprocessing)


Welcome to JupyROOT 6.13/02


<module 'postprocessing_utils' from '/home/ovtin/cernbox/HHggbb/HHbbgg_ETH_devel/Training/python/postprocessing_utils.pyc'>

In [2]:
ntuples = '2016'
# "%" sign allows to interpret the rest as a system command
%env data=$utils.IO.ldata$ntuples
files = ! ls $data | sort -t_ -k 3 -n
#signal = [s for s in files if "output_GluGluToHHTo2B2G_node_SM_13TeV-madgraph_2016_ptmjj.root" in s]
#bkgr = [s for s in files if "DoubleEG_2016_ptmjj.root" in s]
signal = [s for s in files if "output_GluGluToHHTo2B2G_node_SM_13TeV-madgraph_2016.root" in s]
bkgr = [s for s in files if "DoubleEG_2016.root" in s]

utils.IO.add_signal(ntuples,signal,1)
utils.IO.add_background(ntuples,bkgr,-1)

ntuples = '2017'
# "%" sign allows to interpret the rest as a system command
%env data=$utils.IO.ldata$ntuples
files = ! ls $data | sort -t_ -k 3 -n
#signal = [s for s in files if "output_GluGluToHHTo2B2G_node_SM_13TeV-madgraph_2017_ptmjj.root" in s]
#bkgr = [s for s in files if "DoubleEG_2017_ptmjj.root" in s]
signal = [s for s in files if "output_GluGluToHHTo2B2G_node_SM_13TeV-madgraph_2017.root" in s]
bkgr = [s for s in files if "DoubleEG_2017.root" in s]
utils.IO.add_signal(ntuples,signal,-1)
utils.IO.add_background(ntuples,bkgr,-1)


for i in range(len(utils.IO.backgroundName)):        
    print "using background file n."+str(i)+": "+utils.IO.backgroundName[i]
for i in range(len(utils.IO.signalName)):    
    print "using signal file n."+str(i)+": "+utils.IO.signalName[i]

env: data=/home/ovtin/cernbox/HHggbb/HHbbgg_ETH_devel/Training/2016
env: data=/home/ovtin/cernbox/HHggbb/HHbbgg_ETH_devel/Training/2017
using background file n.0: /home/ovtin/cernbox/HHggbb/HHbbgg_ETH_devel/Training/2016/DoubleEG_2016.root
using background file n.1: /home/ovtin/cernbox/HHggbb/HHbbgg_ETH_devel/Training/2017/DoubleEG_2017.root
using signal file n.0: /home/ovtin/cernbox/HHggbb/HHbbgg_ETH_devel/Training/2016/output_GluGluToHHTo2B2G_node_SM_13TeV-madgraph_2016.root
using signal file n.1: /home/ovtin/cernbox/HHggbb/HHbbgg_ETH_devel/Training/2017/output_GluGluToHHTo2B2G_node_SM_13TeV-madgraph_2017.root


In [3]:
#use noexpand for root expressions, it needs this file https://github.com/ibab/root_pandas/blob/master/root_pandas/readwrite.py
#standart of input values 
branch_names = 'absCosThetaStar_CS,absCosTheta_bb,absCosTheta_gg,PhoJetMinDr,leadingJet_DeepCSV,subleadingJet_DeepCSV,leadingPhotonSigOverE,subleadingPhotonSigOverE,sigmaMOverM,diphotonCandidatePtOverdiHiggsM,dijetCandidatePtOverdiHiggsM,noexpand:leadingJet_bRegNNResolution/leadingJet_pt,noexpand:subleadingJet_bRegNNResolution/subleadingJet_pt,noexpand:sigmaMJets/Mjj,rho'.split(",")
#st values with adding pt_gg/m_gg
#branch_names = 'absCosThetaStar_CS,absCosTheta_bb,absCosTheta_gg,PhoJetMinDr,leadingJet_DeepCSV,subleadingJet_DeepCSV,leadingPhotonSigOverE,subleadingPhotonSigOverE,sigmaMOverM,diphotonCandidatePtOverdiHiggsM,dijetCandidatePtOverdiHiggsM,noexpand:leadingJet_bRegNNResolution/leadingJet_pt,noexpand:subleadingJet_bRegNNResolution/subleadingJet_pt,noexpand:sigmaMJets/Mjj,rho,noexpand:leadingPhoton_pt/CMS_hgg_mass,noexpand:subleadingPhoton_pt/CMS_hgg_mass'.split(",")
#st values with adding pt_gg/m_gg, pt_jj/M_jj
#branch_names = 'absCosThetaStar_CS,absCosTheta_bb,absCosTheta_gg,PhoJetMinDr,leadingJet_DeepCSV,subleadingJet_DeepCSV,leadingPhotonSigOverE,subleadingPhotonSigOverE,sigmaMOverM,diphotonCandidatePtOverdiHiggsM,dijetCandidatePtOverdiHiggsM,noexpand:leadingJet_bRegNNResolution/leadingJet_pt,noexpand:subleadingJet_bRegNNResolution/subleadingJet_pt,noexpand:sigmaMJets/Mjj,rho,noexpand:leadingPhoton_pt/CMS_hgg_mass,noexpand:subleadingPhoton_pt/CMS_hgg_mass,noexpand:leadingJet_pt/Mjj,noexpand:subleadingJet_pt/Mjj,PhoJetotherDr'.split(",")

branch_names = [c.strip() for c in branch_names]
#branch_names = (b.replace(" ", "_") for b in branch_names)
#branch_names = list(b.replace("-", "_") for b in branch_names)
print branch_names

import pandas as pd
import root_pandas as rpd
from root_numpy import root2array, list_trees

#print list_trees('/home/ovtin/cernbox/HHggbb/HHbbgg_ETH_devel/Training/2016/output_GluGluToHHTo2B2G_node_SM_13TeV-madgraph.root')
for i in range(len(utils.IO.backgroundName)):        
    print list_trees(utils.IO.backgroundName[i])
        
preprocessing.set_signals("GluGluToHHTo2B2G_node_SM_13TeV_madgraph_13TeV_DoubleHTag_0",branch_names,True)
preprocessing.set_backgrounds("Data_13TeV_DoubleHTag_0",branch_names,True) 
X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig=preprocessing.set_variables(branch_names)

#relative weighting between components of one class is kept, all classes normalized to the same
#weights_sig=preprocessing.weight_signal_with_resolution(weights_sig,y_sig)
weights_bkg,weights_sig=preprocessing.normalize_process_weights(weights_bkg,y_bkg,weights_sig,y_sig)

X_bkg,y_bkg,weights_bkg = preprocessing.randomize(X_bkg,y_bkg,weights_bkg)
X_sig,y_sig,weights_sig = preprocessing.randomize(X_sig,y_sig,weights_sig)

print X_bkg.shape
print y_bkg.shape
#bbggTrees have by default signal and CR events, let's be sure that we clean it
X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig=preprocessing.clean_signal_events(X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig)
print X_bkg.shape
print y_bkg.shape

y_total_train = preprocessing.get_total_training_sample(y_sig,y_bkg).ravel()
X_total_train = preprocessing.get_total_training_sample(X_sig,X_bkg)

y_total_test = preprocessing.get_total_test_sample(y_sig,y_bkg).ravel()
X_total_test = preprocessing.get_total_test_sample(X_sig,X_bkg)

w_total_train = preprocessing.get_total_training_sample(weights_sig,weights_bkg).ravel()
w_total_test = preprocessing.get_total_test_sample(weights_sig,weights_bkg).ravel()

['absCosThetaStar_CS', 'absCosTheta_bb', 'absCosTheta_gg', 'PhoJetMinDr', 'leadingJet_DeepCSV', 'subleadingJet_DeepCSV', 'leadingPhotonSigOverE', 'subleadingPhotonSigOverE', 'sigmaMOverM', 'diphotonCandidatePtOverdiHiggsM', 'dijetCandidatePtOverdiHiggsM', 'noexpand:leadingJet_bRegNNResolution/leadingJet_pt', 'noexpand:subleadingJet_bRegNNResolution/subleadingJet_pt', 'noexpand:sigmaMJets/Mjj', 'rho']
['Data_13TeV_DoubleHTag_0']
['Data_13TeV_DoubleHTag_0']
using tree:GluGluToHHTo2B2G_node_SM_13TeV_madgraph_13TeV_DoubleHTag_0
(508299, 15)
(508299, 1)
(508299, 15)
(508299, 1)


In [4]:
import xgboost as xgb
clf = xgb.XGBClassifier(max_depth=5,learning_rate=0.1,n_estimators=50, min_child_weight=1e-5, nthread= 12)

In [5]:
reload(utils)
import preprocessing_utils as preprocessing
reload(preprocessing)
import plotting_utils as plotting
reload(plotting)
import optimization_utils as optimization
reload(optimization)
import postprocessing_utils as postprocessing
reload(postprocessing)
import pandas as pd
#param_grid = {"n_estimators": [1000,1500],
#              "max_depth": [3,4,5],                                                                                                                                                                                                
#              'reg_lambda':[0.1, 1]
#              }
#param_grid = {"n_estimators": [60,80],
#              "max_depth": [3,4]                                                                                                                                                                                                
#              }
param_grid = {"n_estimators": [700,1500,2000],
              "max_depth": [3, 4, 5, 8],                                                                                                                                            
              'learning_rate': [0.01, 0.1],
             } 
#optimization.optimize_parameters_randomizedCV(clf,X_total_train,y_total_train,param_grid,cvOpt=5,nIter=500,weights=w_total_train)
#optimization.optimize_parameters_randomizedCV(clf,X_total_train,y_total_train,param_grid,cvOpt=3,weights=w_total_train)
optimization.optimize_parameters_randomizedCV(clf,X_total_train,y_total_train,param_grid,cvOpt=3)


=====Optimization with randomized search cv=====
-Initial Accuracy-
Accuracy: 0.91768 (+/- 0.08310)
Fitting 3 folds for each of 4 candidates, totalling 12 fits


/home/ovtin/.local/lib/python2.7/site-packages/sklearn/model_selection/_search.py:271: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   6 out of  12 | elapsed:  1.1min remaining:  1.1min
[Parallel(n_jobs=10)]: Done  12 out of  12 | elapsed:  1.4min finished


Best parameter set found on development set:

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1e-05, missing=None, n_estimators=80,
       n_jobs=1, nthread=12, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

Grid scores on a subset of the development set:

                                   params  mean_train_score  std_train_score  \
0  {u'n_estimators': 60, u'max_depth': 3}          0.968552         0.001040   
1  {u'n_estimators': 80, u'max_depth': 3}          0.973408         0.000703   
2  {u'n_estimators': 60, u'max_depth': 4}          0.975212         0.000656   
3  {u'n_estimators': 80, u'max_depth': 4}          0.978777         0.000593   

   mean_test_score  std_test_score  
0         0.965645        0.002389  
1         0.970031        0.001948  
2  

/home/ovtin/.local/lib/python2.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('std_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ovtin/.local/lib/python2.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ovtin/.local/lib/python2.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/ovtin/.local/lib/python2.7/site-packages

{'mean_fit_time': array([24.85299476, 47.06372301, 50.12661497, 56.74168833]),
 'mean_score_time': array([0.56259863, 0.97124664, 0.55741302, 0.74316295]),
 'mean_test_score': array([0.96564518, 0.97003108, 0.9702306 , 0.97281843]),
 'mean_train_score': array([0.96855181, 0.973408  , 0.97521188, 0.97877654]),
 'param_max_depth': masked_array(data=[3, 3, 4, 4],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[60, 80, 60, 80],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 3, 'n_estimators': 60},
  {'max_depth': 3, 'n_estimators': 80},
  {'max_depth': 4, 'n_estimators': 60},
  {'max_depth': 4, 'n_estimators': 80}],
 'rank_test_score': array([4, 3, 2, 1], dtype=int32),
 'split0_test_score': array([0.96885498, 0.97246466, 0.9736189 , 0.97528787]),
 'split0_train_score': array([0.96721649, 0.97260095, 0.97435845, 0.9781